# Model Context Protocol (MCP) Servers - Building AI Travel Agents

In this notebook, we'll explore the **Model Context Protocol (MCP)** - a powerful standard for connecting AI assistants to external data sources and tools. We'll build a comprehensive travel booking system that demonstrates:

1. **MCP Server Implementation** - Using FastMCP to create a travel booking server similar to Amadeus
2. **AI Travel Agent** - An intelligent agent that uses the MCP server for flight searches and bookings
3. **Supervisor Agent** - A coordinating agent that manages multiple travel-related tasks
4. **Integration** - How MCP servers connect with LangGraph agents for complex workflows

## What is MCP?

The Model Context Protocol (MCP) is an open standard that enables AI assistants to securely connect to external data sources and tools. Instead of requiring custom integrations for each data source, MCP provides:

- **Standardized Communication** - Uniform way for AI models to interact with external services
- **Security** - Controlled access to sensitive data and operations
- **Extensibility** - Easy addition of new capabilities through MCP servers
- **Tool Integration** - Native support for function calling and tool usage

## Architecture Overview

Our travel booking system will consist of:

```
┌─────────────────────┐    ┌─────────────────────┐    ┌─────────────────────┐
│   Supervisor        │    │   Travel Agent      │    │   MCP Server        │
│   Agent             │◄──►│   (LangGraph)       │◄──►│   (FastMCP)         │
│                     │    │                     │    │                     │
│ - Route requests    │    │ - Flight search     │    │ - Flight API        │
│ - Coordinate tasks  │    │ - Booking logic     │    │ - Hotel API         │
│ - Manage workflow   │    │ - Price comparison  │    │ - Car rental API    │
└─────────────────────┘    └─────────────────────┘    └─────────────────────┘
```

## Setup and Imports

In [1]:
import os
import json
import uuid
import asyncio
import logging
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Any
from pydantic import BaseModel, Field

# FastMCP for MCP server implementation
from fastmcp import FastMCP
from fastmcp.tools import Tool
from fastmcp.resources import Resource

# LangChain and LangGraph for AI agents
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict, Annotated

# Load environment variables
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Display utilities
from IPython.display import Image, Markdown, display
import re

def printmd(string):
    clean_content = re.sub(r'^```markdown\n', '', string)
    clean_content = re.sub(r'^```\n', '', clean_content)
    clean_content = re.sub(r'\n```$', '', clean_content)
    clean_content = clean_content.replace('$', r'\$')
    display(Markdown(clean_content))

In [2]:
# Configure Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

COMPLETION_TOKENS = 2000
llm = AzureChatOpenAI(
    deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
    temperature=0, 
    max_tokens=COMPLETION_TOKENS, 
    streaming=True
)

## Part 1: Building the MCP Travel Server with FastMCP

We'll start by creating an MCP server that provides travel booking capabilities similar to what you'd find in services like Amadeus. Our server will offer:

- Flight search and booking
- Hotel search and booking  
- Car rental search and booking
- Trip management

### Data Models

First, let's define the data structures for our travel services:

In [3]:
# Data models for travel services
class FlightSearchRequest(BaseModel):
    origin: str = Field(description="Origin airport code (e.g., 'JFK')")
    destination: str = Field(description="Destination airport code (e.g., 'LAX')")
    departure_date: str = Field(description="Departure date in YYYY-MM-DD format")
    return_date: Optional[str] = Field(None, description="Return date for round trip")
    passengers: int = Field(1, description="Number of passengers")
    class_type: str = Field("economy", description="Flight class: economy, business, first")

class Flight(BaseModel):
    flight_id: str
    airline: str
    flight_number: str
    origin: str
    destination: str
    departure_time: str
    arrival_time: str
    duration: str
    price: float
    currency: str = "USD"
    class_type: str
    available_seats: int

class HotelSearchRequest(BaseModel):
    location: str = Field(description="City or area to search for hotels")
    check_in: str = Field(description="Check-in date in YYYY-MM-DD format")
    check_out: str = Field(description="Check-out date in YYYY-MM-DD format")
    guests: int = Field(1, description="Number of guests")
    rooms: int = Field(1, description="Number of rooms")

class Hotel(BaseModel):
    hotel_id: str
    name: str
    location: str
    star_rating: int
    price_per_night: float
    currency: str = "USD"
    amenities: List[str]
    available_rooms: int
    review_score: float

class CarRentalRequest(BaseModel):
    location: str = Field(description="Pickup location")
    pickup_date: str = Field(description="Pickup date in YYYY-MM-DD format")
    return_date: str = Field(description="Return date in YYYY-MM-DD format")
    car_type: str = Field("economy", description="Car type: economy, compact, midsize, luxury")

class CarRental(BaseModel):
    rental_id: str
    company: str
    car_model: str
    car_type: str
    location: str
    price_per_day: float
    currency: str = "USD"
    features: List[str]
    available: bool

class BookingRequest(BaseModel):
    item_id: str = Field(description="ID of the flight, hotel, or car to book")
    item_type: str = Field(description="Type of booking: flight, hotel, car")
    customer_info: Dict[str, Any] = Field(description="Customer information")

class Booking(BaseModel):
    booking_id: str
    item_type: str
    item_id: str
    customer_info: Dict[str, Any]
    status: str
    booking_date: str
    total_price: float
    currency: str = "USD"

print("✅ Data models defined successfully")

✅ Data models defined successfully


## 🔄 **Real Amadeus API Integration - Step by Step**

This section demonstrates the complete integration of a **real Amadeus Travel API** with our MCP server, replacing all mock data with authentic travel information.

### **What We're Implementing:**

1. **🔐 Real API Authentication**
   - OAuth2 client credentials flow
   - Automatic token refresh mechanism
   - Secure credential management from `credentials.env`

2. **🌐 Live Amadeus API Client**
   - Direct connection to Amadeus Test API
   - Real-time flight search and pricing
   - Actual hotel availability and rates
   - Authentic airport and airline data


### **Key Implementation Steps:**

**Step 1: Amadeus Client Setup**
- Configure API credentials from environment variables
- Implement OAuth2 token management
- Create authenticated request handler
- Add comprehensive error handling

**Step 2: Setup MCP Server with tools based off the Amadeus Client**
- `search_flights`: Real flight search with Amadeus Flight Offers API
- `search_hotels`: Live hotel search with current availability
- `search_airports`: IATA airport lookup with coordinates
- `get_airline_info`: Official airline information

**Step 3: Production-Ready Features**
- Rate limiting awareness
- Comprehensive error handling
- Logging and monitoring
- Security best practices

In [4]:
# ================== Real Amadeus API Client Implementation ==================
# Replace mock data with actual Amadeus API integration

import requests
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Load environment variables
load_dotenv("credentials.env")

# Amadeus API Configuration
AMADEUS_API_KEY = os.getenv("AMADEUS_API_KEY")
AMADEUS_API_SECRET = os.getenv("AMADEUS_API_SECRET")
AMADEUS_BASE_URL = "https://api.amadeus.com"
AMADEUS_TEST_BASE_URL = "https://test.api.amadeus.com"  # Use test environment

# We'll use the test environment for development
BASE_URL = AMADEUS_TEST_BASE_URL

class AmadeusClient:
    """Client for interacting with Amadeus API"""

    def __init__(self):
        self.api_key = "5SK7r1YbvPacpnprO9pJt4VGEpwaSQZO"
        self.api_secret = "dmavJiUlrGIbMZTh"
        self.base_url = BASE_URL
        self.access_token = None
        self.token_expiry = None

    def get_access_token(self):
        """Get or refresh access token for Amadeus API"""
        # Check if token is still valid
        if self.access_token and self.token_expiry:
            if datetime.now() < self.token_expiry:
                return self.access_token

        # Get new token
        url = f"{self.base_url}/v1/security/oauth2/token"
        headers = {
            "Content-Type": "application/x-www-form-urlencoded"
        }
        data = {
            "grant_type": "client_credentials",
            "client_id": self.api_key,
            "client_secret": self.api_secret
        }

        try:
            response = requests.post(url, headers=headers, data=data)
            response.raise_for_status()

            token_data = response.json()
            self.access_token = token_data["access_token"]
            # Set expiry with 5 minute buffer
            expires_in = token_data.get("expires_in", 1799)
            self.token_expiry = datetime.now() + timedelta(seconds=expires_in - 300)

            logger.info("Successfully obtained Amadeus access token")
            return self.access_token

        except requests.exceptions.RequestException as e:
            logger.error(f"Failed to get access token: {e}")
            raise

    def make_request(self, endpoint: str, method: str = "GET", params: Dict = None, data: Dict = None):
        """Make authenticated request to Amadeus API"""
        token = self.get_access_token()

        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        }

        url = f"{self.base_url}{endpoint}"

        try:
            if method == "GET":
                response = requests.get(url, headers=headers, params=params)
            elif method == "POST":
                response = requests.post(url, headers=headers, json=data)
            else:
                raise ValueError(f"Unsupported HTTP method: {method}")

            response.raise_for_status()
            return response.json()

        except requests.exceptions.RequestException as e:
            logger.error(f"API request failed: {e}")
            if hasattr(e, 'response') and e.response is not None:
                logger.error(f"Response content: {e.response.text}")
            raise

# Initialize Amadeus client
amadeus_client = AmadeusClient()

print("✅ Real Amadeus API client initialized successfully")
print(f"Environment: {'TEST' if 'test' in BASE_URL else 'PRODUCTION'}")
print(f"API Key: {AMADEUS_API_KEY[:10]}..." if AMADEUS_API_KEY else "❌ No API Key found")     

# Test connection
try:
    token = amadeus_client.get_access_token()
    print("✅ Successfully connected to Amadeus API")
    print(f"Token obtained: {token[:20]}...")
    print(f"Base URL: {BASE_URL}")
except Exception as e:
    print(f"❌ Failed to connect to Amadeus API: {e}")
    print("Please check your credentials in credentials.env file")
    print("Required variables: AMADEUS_API_KEY, AMADEUS_API_SECRET")

✅ Real Amadeus API client initialized successfully
Environment: TEST
API Key: 5SK7r1YbvP...


INFO:__main__:Successfully obtained Amadeus access token


✅ Successfully connected to Amadeus API
Token obtained: il28c7CK1FWgSAuKGgAI...
Base URL: https://test.api.amadeus.com


### FastMCP Server Implementation

Now we'll create the MCP server using FastMCP. This server will expose our travel booking capabilities as standardized MCP tools:

In [5]:
# Real Amadeus MCP Server Implementation
# Create the FastMCP server with real Amadeus API integration

# Create the FastMCP server
mcp_server = FastMCP(name="amadeus-travel-server")

# ================== MCP Tool Implementations ==================

@mcp_server.tool()
def search_flights(
    origin: str,
    destination: str,
    departure_date: str,
    adults: int = 1,
    children: int = 0,
    infants: int = 0,
    travel_class: str = "ECONOMY",
    return_date: Optional[str] = None,
    nonstop: bool = False,
    currency: str = "USD",
    max_results: int = 10
) -> str:
    """Search for flights using Amadeus Flight Offers Search API.
    
    Args:
        origin: Origin airport IATA code (e.g., 'JFK', 'LAX')
        destination: Destination airport IATA code
        departure_date: Departure date in YYYY-MM-DD format
        adults: Number of adult passengers (12+ years)
        children: Number of child passengers (2-11 years)
        infants: Number of infant passengers (under 2 years)
        travel_class: Travel class - ECONOMY, PREMIUM_ECONOMY, BUSINESS, or FIRST
        return_date: Return date for round trip (optional, YYYY-MM-DD format)
        nonstop: Only show nonstop flights if True
        currency: Currency code for prices (default: USD)
        max_results: Maximum number of results to return (default: 10)
    
    Returns:
        JSON string with flight search results including prices, times, and availability
    """
    try:
        # Build query parameters
        params = {
            "originLocationCode": origin.upper(),
            "destinationLocationCode": destination.upper(),
            "departureDate": departure_date,
            "adults": adults,
            "currencyCode": currency,
            "max": min(max_results, 250)  # Amadeus max is 250
        }
        
        # Add optional parameters
        if return_date:
            params["returnDate"] = return_date
        
        if children > 0:
            params["children"] = children
            
        if infants > 0:
            params["infants"] = infants
        
        if travel_class != "ECONOMY":
            params["travelClass"] = travel_class
            
        if nonstop:
            params["nonStop"] = "true"
        
        # Make API request
        endpoint = "/v2/shopping/flight-offers"
        response = amadeus_client.make_request(endpoint, params=params)
        
        # Process and format results
        flights = response.get("data", [])
        
        if not flights:
            return json.dumps({
                "success": False,
                "message": "No flights found for the specified criteria",
                "search_params": params
            }, indent=2)
        
        # Format flight results
        formatted_flights = []
        for flight_offer in flights[:max_results]:
            # Extract key information
            price = flight_offer.get("price", {})
            itineraries = flight_offer.get("itineraries", [])
            
            formatted_flight = {
                "id": flight_offer.get("id"),
                "price": {
                    "total": price.get("total"),
                    "currency": price.get("currency", "USD"),
                    "base": price.get("base"),
                    "fees": price.get("fees", [])
                },
                "itineraries": []
            }
            
            # Process each itinerary (outbound and return)
            for itinerary in itineraries:
                segments = itinerary.get("segments", [])
                
                itinerary_info = {
                    "duration": itinerary.get("duration"),
                    "segments": []
                }
                
                for segment in segments:
                    segment_info = {
                        "departure": {
                            "airport": segment.get("departure", {}).get("iataCode"),
                            "terminal": segment.get("departure", {}).get("terminal"),
                            "at": segment.get("departure", {}).get("at")
                        },
                        "arrival": {
                            "airport": segment.get("arrival", {}).get("iataCode"),
                            "terminal": segment.get("arrival", {}).get("terminal"),
                            "at": segment.get("arrival", {}).get("at")
                        },
                        "carrier": segment.get("carrierCode"),
                        "flight_number": segment.get("number"),
                        "aircraft": segment.get("aircraft", {}).get("code"),
                        "duration": segment.get("duration"),
                        "cabin": segment.get("cabin")
                    }
                    itinerary_info["segments"].append(segment_info)
                
                formatted_flight["itineraries"].append(itinerary_info)
            
            formatted_flights.append(formatted_flight)
        
        result = {
            "success": True,
            "count": len(formatted_flights),
            "search_criteria": {
                "origin": origin,
                "destination": destination,
                "departure_date": departure_date,
                "return_date": return_date,
                "passengers": {
                    "adults": adults,
                    "children": children,
                    "infants": infants
                },
                "travel_class": travel_class
            },
            "flights": formatted_flights
        }
        
        return json.dumps(result, indent=2)
        
    except Exception as e:
        logger.error(f"Flight search failed: {e}")
        return json.dumps({
            "success": False,
            "error": str(e),
            "message": "Failed to search flights. Please check your parameters and try again."
        }, indent=2)

@mcp_server.tool()
def search_hotels(
    city_code: str,
    check_in: str,
    check_out: str,
    adults: int = 1,
    room_quantity: int = 1,
    radius: int = 5,
    radius_unit: str = "KM",
    amenities: Optional[List[str]] = None,
    ratings: Optional[List[int]] = None,
    hotel_name: Optional[str] = None,
    price_range: Optional[str] = None,
    currency: str = "USD"
) -> str:
    """Search for hotels using Amadeus Hotel Search API.
    
    Args:
        city_code: City IATA code (e.g., 'NYC', 'PAR', 'LON')
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        adults: Number of adult guests
        room_quantity: Number of rooms needed
        radius: Search radius from city center (default: 5)
        radius_unit: Unit for radius - KM or MILE (default: KM)
        amenities: List of required amenities (e.g., ['WIFI', 'PARKING', 'POOL'])
        ratings: List of acceptable star ratings (1-5)
        hotel_name: Search for specific hotel name (partial match)
        price_range: Price range filter (e.g., '50-200')
        currency: Currency code for prices (default: USD)
    
    Returns:
        JSON string with hotel search results including prices, amenities, and availability
    """
    try:
        # First, get hotels by city
        params = {
            "cityCode": city_code.upper(),
            "radius": radius,
            "radiusUnit": radius_unit
        }
        
        if ratings:
            params["ratings"] = ",".join(map(str, ratings))
        
        if amenities:
            params["amenities"] = ",".join(amenities)
        
        if hotel_name:
            params["hotelName"] = hotel_name
        
        # Get list of hotels
        endpoint = "/v1/reference-data/locations/hotels/by-city"
        hotels_response = amadeus_client.make_request(endpoint, params=params)
        
        hotels = hotels_response.get("data", [])
        
        if not hotels:
            return json.dumps({
                "success": False,
                "message": f"No hotels found in {city_code}",
                "search_params": params
            }, indent=2)
        
        # Get hotel IDs (limit to first 20 hotels)
        hotel_ids = [hotel["hotelId"] for hotel in hotels[:20]]
        
        # Now get hotel offers for these hotels
        offers_params = {
            "hotelIds": ",".join(hotel_ids),
            "checkInDate": check_in,
            "checkOutDate": check_out,
            "adults": adults,
            "roomQuantity": room_quantity,
            "currency": currency,
            "bestRateOnly": "true"
        }
        
        if price_range:
            price_parts = price_range.split("-")
            if len(price_parts) == 2:
                offers_params["priceRange"] = price_range
        
        # Get hotel offers
        endpoint = "/v3/shopping/hotel-offers"
        offers_response = amadeus_client.make_request(endpoint, params=offers_params)
        
        hotel_offers = offers_response.get("data", [])
        
        # Format results
        formatted_hotels = []
        for offer in hotel_offers:
            hotel = offer.get("hotel", {})
            offers = offer.get("offers", [])
            
            if offers:
                best_offer = offers[0]  # Take the best rate
                
                formatted_hotel = {
                    "hotel_id": hotel.get("hotelId"),
                    "name": hotel.get("name"),
                    "chain_code": hotel.get("chainCode"),
                    "city_code": hotel.get("cityCode"),
                    "latitude": hotel.get("latitude"),
                    "longitude": hotel.get("longitude"),
                    "rating": hotel.get("rating"),
                    "amenities": hotel.get("amenities", []),
                    "description": hotel.get("description", {}),
                    "best_offer": {
                        "id": best_offer.get("id"),
                        "check_in": best_offer.get("checkInDate"),
                        "check_out": best_offer.get("checkOutDate"),
                        "room": {
                            "type": best_offer.get("room", {}).get("type"),
                            "category": best_offer.get("room", {}).get("typeEstimated", {}).get("category"),
                            "beds": best_offer.get("room", {}).get("typeEstimated", {}).get("beds"),
                            "bed_type": best_offer.get("room", {}).get("typeEstimated", {}).get("bedType"),
                            "description": best_offer.get("room", {}).get("description", {}).get("text")
                        },
                        "guests": best_offer.get("guests", {}),
                        "price": {
                            "total": best_offer.get("price", {}).get("total"),
                            "currency": best_offer.get("price", {}).get("currency"),
                            "base": best_offer.get("price", {}).get("base"),
                            "taxes": best_offer.get("price", {}).get("taxes")
                        },
                        "policies": best_offer.get("policies", {})
                    }
                }
                formatted_hotels.append(formatted_hotel)
        
        result = {
            "success": True,
            "count": len(formatted_hotels),
            "search_criteria": {
                "city_code": city_code,
                "check_in": check_in,
                "check_out": check_out,
                "adults": adults,
                "rooms": room_quantity
            },
            "hotels": formatted_hotels
        }
        
        return json.dumps(result, indent=2)
        
    except Exception as e:
        logger.error(f"Hotel search failed: {e}")
        return json.dumps({
            "success": False,
            "error": str(e),
            "message": "Failed to search hotels. Please check your parameters and try again."
        }, indent=2)

@mcp_server.tool()
def get_flight_price(flight_offer: str) -> str:
    """Get confirmed price for a specific flight offer.
    
    Args:
        flight_offer: The flight offer data (as JSON string) from search_flights
    
    Returns:
        JSON string with confirmed pricing details
    """
    try:
        # Parse the flight offer
        if isinstance(flight_offer, str):
            offer_data = json.loads(flight_offer)
        else:
            offer_data = flight_offer
        
        # Make pricing request
        endpoint = "/v1/shopping/flight-offers/pricing"
        
        pricing_request = {
            "data": {
                "type": "flight-offers-pricing",
                "flightOffers": [offer_data]
            }
        }
        
        response = amadeus_client.make_request(endpoint, method="POST", data=pricing_request)
        
        result = {
            "success": True,
            "pricing": response.get("data", {})
        }
        
        return json.dumps(result, indent=2)
        
    except Exception as e:
        logger.error(f"Flight pricing failed: {e}")
        return json.dumps({
            "success": False,
            "error": str(e),
            "message": "Failed to get flight pricing."
        }, indent=2)

@mcp_server.tool()
def create_flight_booking(
    flight_offer: str,
    travelers: List[Dict[str, Any]],
    contact: Dict[str, Any]
) -> str:
    """Create a flight booking (in test environment only).
    
    Args:
        flight_offer: The confirmed flight offer from get_flight_price
        travelers: List of traveler information dictionaries with:
            - id: Traveler ID (e.g., "1")
            - dateOfBirth: Date of birth (YYYY-MM-DD)
            - name: {firstName: str, lastName: str}
            - gender: MALE or FEMALE
            - contact: {emailAddress: str, phones: [{deviceType: str, countryCallingCode: str, number: str}]}
            - documents: [{documentType: "PASSPORT", birthPlace: str, issuanceLocation: str, issuanceDate: str, number: str, expiryDate: str, issuanceCountry: str, validityCountry: str, nationality: str, holder: bool}]
        contact: Contact information for the booking
    
    Returns:
        JSON string with booking confirmation
    """
    try:
        # Note: In test environment, this will create a test booking
        endpoint = "/v1/booking/flight-orders"
        
        booking_request = {
            "data": {
                "type": "flight-order",
                "flightOffers": [json.loads(flight_offer) if isinstance(flight_offer, str) else flight_offer],
                "travelers": travelers,
                "remarks": {
                    "general": [
                        {
                            "subType": "GENERAL_MISCELLANEOUS",
                            "text": "Booking created via MCP Travel Server"
                        }
                    ]
                },
                "ticketingAgreement": {
                    "option": "DELAY_TO_CANCEL",
                    "delay": "6D"
                },
                "contacts": [contact]
            }
        }
        
        response = amadeus_client.make_request(endpoint, method="POST", data=booking_request)
        
        result = {
            "success": True,
            "booking": response.get("data", {}),
            "warnings": response.get("warnings", [])
        }
        
        return json.dumps(result, indent=2)
        
    except Exception as e:
        logger.error(f"Flight booking failed: {e}")
        return json.dumps({
            "success": False,
            "error": str(e),
            "message": "Failed to create flight booking. This might be due to test environment limitations."
        }, indent=2)

@mcp_server.tool()
def search_airports(
    keyword: str,
    country_code: Optional[str] = None
) -> str:
    """Search for airports by keyword or location.
    
    Args:
        keyword: Search keyword (city name, airport name, or code)
        country_code: Optional 2-letter country code to filter results
    
    Returns:
        JSON string with matching airports
    """
    try:
        params = {
            "keyword": keyword,
            "max": 10
        }
        
        if country_code:
            params["countryCode"] = country_code.upper()
        
        endpoint = "/v1/reference-data/locations"
        params["subType"] = "AIRPORT"
        
        response = amadeus_client.make_request(endpoint, params=params)
        
        airports = response.get("data", [])
        
        formatted_airports = []
        for airport in airports:
            formatted_airports.append({
                "iata_code": airport.get("iataCode"),
                "name": airport.get("name"),
                "city": airport.get("address", {}).get("cityName"),
                "country": airport.get("address", {}).get("countryName"),
                "country_code": airport.get("address", {}).get("countryCode"),
                "latitude": airport.get("geoCode", {}).get("latitude"),
                "longitude": airport.get("geoCode", {}).get("longitude"),
                "timezone": airport.get("timeZoneOffset")
            })
        
        result = {
            "success": True,
            "count": len(formatted_airports),
            "airports": formatted_airports
        }
        
        return json.dumps(result, indent=2)
        
    except Exception as e:
        logger.error(f"Airport search failed: {e}")
        return json.dumps({
            "success": False,
            "error": str(e),
            "message": "Failed to search airports."
        }, indent=2)

@mcp_server.tool()
def get_city_info(
    keyword: str
) -> str:
    """Get information about a city for travel planning.
    
    Args:
        keyword: City name or IATA code
    
    Returns:
        JSON string with city information including IATA code
    """
    try:
        params = {
            "keyword": keyword,
            "max": 5,
            "subType": "CITY"
        }
        
        endpoint = "/v1/reference-data/locations"
        response = amadeus_client.make_request(endpoint, params=params)
        
        cities = response.get("data", [])
        
        if not cities:
            return json.dumps({
                "success": False,
                "message": f"No city found matching '{keyword}'"
            }, indent=2)
        
        formatted_cities = []
        for city in cities:
            formatted_cities.append({
                "iata_code": city.get("iataCode"),
                "name": city.get("name"),
                "country": city.get("address", {}).get("countryName"),
                "country_code": city.get("address", {}).get("countryCode"),
                "state_code": city.get("address", {}).get("stateCode"),
                "latitude": city.get("geoCode", {}).get("latitude"),
                "longitude": city.get("geoCode", {}).get("longitude"),
                "timezone": city.get("timeZoneOffset")
            })
        
        result = {
            "success": True,
            "count": len(formatted_cities),
            "cities": formatted_cities
        }
        
        return json.dumps(result, indent=2)
        
    except Exception as e:
        logger.error(f"City info search failed: {e}")
        return json.dumps({
            "success": False,
            "error": str(e),
            "message": "Failed to get city information."
        }, indent=2)

@mcp_server.tool()
def get_airline_info(carrier_code: str) -> str:
    """Get information about an airline by its carrier code.
    
    Args:
        carrier_code: Airline carrier code (e.g., 'AA', 'DL', 'UA')
    
    Returns:
        JSON string with airline information
    """
    try:
        endpoint = f"/v1/reference-data/airlines"
        params = {"airlineCodes": carrier_code.upper()}
        
        response = amadeus_client.make_request(endpoint, params=params)
        
        airlines = response.get("data", [])
        
        if not airlines:
            return json.dumps({
                "success": False,
                "message": f"No airline found with code '{carrier_code}'"
            }, indent=2)
        
        airline = airlines[0]
        
        result = {
            "success": True,
            "airline": {
                "iata_code": airline.get("iataCode"),
                "icao_code": airline.get("icaoCode"),
                "business_name": airline.get("businessName"),
                "common_name": airline.get("commonName")
            }
        }
        
        return json.dumps(result, indent=2)
        
    except Exception as e:
        logger.error(f"Airline info search failed: {e}")
        return json.dumps({
            "success": False,
            "error": str(e),
            "message": "Failed to get airline information."
        }, indent=2)

@mcp_server.tool()
def check_flight_status(
    carrier_code: str,
    flight_number: str,
    scheduled_departure_date: str
) -> str:
    """Check the status of a specific flight.
    
    Args:
        carrier_code: Airline carrier code (e.g., 'AA', 'DL')
        flight_number: Flight number (e.g., '100')
        scheduled_departure_date: Scheduled departure date (YYYY-MM-DD)
    
    Returns:
        JSON string with flight status information
    """
    try:
        endpoint = "/v2/schedule/flights"
        params = {
            "carrierCode": carrier_code.upper(),
            "flightNumber": flight_number,
            "scheduledDepartureDate": scheduled_departure_date
        }
        
        response = amadeus_client.make_request(endpoint, params=params)
        
        flights = response.get("data", [])
        
        if not flights:
            return json.dumps({
                "success": False,
                "message": f"No flight found for {carrier_code}{flight_number} on {scheduled_departure_date}"
            }, indent=2)
        
        flight = flights[0]
        
        result = {
            "success": True,
            "flight_status": {
                "carrier": carrier_code,
                "flight_number": flight_number,
                "departure": {
                    "airport": flight.get("flightPoints", [{}])[0].get("iataCode"),
                    "scheduled": flight.get("flightPoints", [{}])[0].get("departure", {}).get("timings", [{}])[0].get("value")
                },
                "arrival": {
                    "airport": flight.get("flightPoints", [{}])[-1].get("iataCode") if flight.get("flightPoints") else None,
                    "scheduled": flight.get("flightPoints", [{}])[-1].get("arrival", {}).get("timings", [{}])[0].get("value") if flight.get("flightPoints") else None
                }
            }
        }
        
        return json.dumps(result, indent=2)
        
    except Exception as e:
        logger.error(f"Flight status check failed: {e}")
        return json.dumps({
            "success": False,
            "error": str(e),
            "message": "Failed to check flight status."
        }, indent=2)


In [ ]:
# ================== Start MCP Server with uvicorn ==================
import threading, time, uvicorn

# Prefer HTTP if available, else fall back to SSE
if hasattr(mcp_server, "http_app"):
    asgi_app = mcp_server.http_app()
    endpoint = "http://127.0.0.1:8000"
elif hasattr(mcp_server, "sse_app"):
    asgi_app = mcp_server.sse_app()
    endpoint = "http://127.0.0.1:8000"  # SSE over HTTP
else:
    raise RuntimeError("This FastMCP version doesn't expose http_app() or sse_app().")

def start_server():
    print("🚀 Starting Amadeus MCP Server (uvicorn)...")
    uvicorn.run(asgi_app, host="127.0.0.1", port=8000, log_level="info")

t = threading.Thread(target=start_server, daemon=True)
t.start()
time.sleep(2)
print(f"✅ MCP Server running at {endpoint}")


c:\Users\Guido\AppData\Local\Programs\Python\Python311\Lib\site-packages\websockets\legacy\__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
c:\Users\Guido\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\protocols\websockets\websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol
INFO:     Started server process [77340]
INFO:     Waiting for application startup.
INFO:mcp.server.streamable_http_manager:StreamableHTTP session manager started
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


🚀 Starting Amadeus MCP Server (uvicorn)...
✅ MCP Server running at http://127.0.0.1:8000


INFO:mcp.server.streamable_http_manager:Created new transport with session ID: 5c6d7d42cb41494ebfc785910ddbade5


INFO:     127.0.0.1:54157 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:54158 - "POST /mcp/ HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:54159 - "GET /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:54160 - "POST /mcp/ HTTP/1.1" 200 OK


INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest
INFO:mcp.server.streamable_http:Terminating session: 5c6d7d42cb41494ebfc785910ddbade5


...

INFO:     127.0.0.1:54190 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:54191 - "POST /mcp/ HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:54192 - "GET /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:54193 - "POST /mcp/ HTTP/1.1" 200 OK


INFO:mcp.server.lowlevel.server:Processing request of type CallToolRequest
ERROR:__main__:Flight search failed: 'MultiServerMCPClient' object has no attribute 'make_request'


INFO:     127.0.0.1:54194 - "POST /mcp/ HTTP/1.1" 200 OK


INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest
INFO:mcp.server.streamable_http:Terminating session: 6822ba89d78346f1952ec2273a9af582


INFO:     127.0.0.1:54195 - "DELETE /mcp/ HTTP/1.1" 200 OK


### Testing the MCP Server

Let's test our MCP Server using an MCP client:

In [7]:
# ================== Probe via FastMCP's own Client (version-safe) ==================
import sys, asyncio
from fastmcp import Client  # FastMCP's client

async def probe():
    # In-memory probe: no network, always compatible
    inmem_client = Client(mcp_server)
    async with inmem_client:
        tools = await inmem_client.list_tools()
        print("📋 Server Components (in-memory probe):")
        print(f"   - Tools: {len(tools)}")

# notebook-safe runner
if "ipykernel" in sys.modules or "PYDEVD_LOAD_VALUES_ASYNC" in sys.modules:
    await probe()
else:
    asyncio.run(probe())

# ================== Amadeus token check ==================
try:
    ok = bool(amadeus_client.get_access_token())
    print(f"   - Amadeus API: {'✅ Connected' if ok else '❌ Failed'}")
except Exception as e:
    print(f"   - Amadeus API: ❌ Error during token check: {e}")

INFO:mcp.server.lowlevel.server:Processing request of type ListToolsRequest


📋 Server Components (in-memory probe):
   - Tools: 8
   - Amadeus API: ✅ Connected


## Part 2: Creating the the AI Travel Agent using MCP

Now we'll create an intelligent travel agent based on an MCP Client that can interact with our MCP server. This agent will:

- Understand natural language travel requests
- Use the MCP server tools to search for flights, hotels, and cars
- Help users make bookings
- Provide travel recommendations

### Create the MCP Client

In [8]:
from langchain_mcp_adapters.client import MultiServerMCPClient

amadeus_client = MultiServerMCPClient({
    "amadeus": {
        "url": "http://127.0.0.1:8000/mcp/",
        "transport": "streamable_http",
    }
})

## Create the AI Travel Agent

In [9]:
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools

from common.prompts import MCP_TRAVEL_AGENT_PROMPT_TEXT, CUSTOM_CHATBOT_PREFIX

async def create_travel_agent(
        llm:AzureChatOpenAI,
        prompt:str,
        name: str
    ):

    tools = await amadeus_client.get_tools()
    travel_agent = create_react_agent(llm, 
                                     tools=tools, 
                                     prompt=prompt,
                                     name=name)
    return travel_agent 

travel_agent = await create_travel_agent(
    llm,
    CUSTOM_CHATBOT_PREFIX + MCP_TRAVEL_AGENT_PROMPT_TEXT,
    "travel_agent"
)

INFO:httpx:HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"
INFO:mcp.client.streamable_http:Received session ID: 5c6d7d42cb41494ebfc785910ddbade5
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-06-18
INFO:httpx:HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"


## QUICK DEMO: Travel Booking with MCP Server
### This demo showcases how to use the Amadeus MCP server to search for real flights

In [10]:
async def stream_graph_updates_async(graph, user_input: str):
    inputs = {"messages": [("human", user_input)]}

    async for event in graph.astream_events(inputs, version="v2"):
        if (event["event"] == "on_chat_model_stream"):
            # Print the content of the chunk progressively
            print(event["data"]["chunk"].content, end="", flush=True)
        elif (event["event"] == "on_tool_start"  ):
            print("\n--")
            print(f"Calling tool: {event['name']} with inputs: {event['data'].get('input')}")
            print("--")

In [11]:
await stream_graph_updates_async(travel_agent, "how much is a flight from Buenos Aires to New York?")

INFO:httpx:HTTP Request: POST https://ksg-openai-swedencentral.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-04-01-preview "HTTP/1.1 200 OK"



--
Calling tool: search_flights with inputs: {'origin': 'EZE', 'destination': 'JFK', 'departure_date': '2023-10-15', 'currency': 'USD'}
--


INFO:httpx:HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"
INFO:mcp.client.streamable_http:Received session ID: 6822ba89d78346f1952ec2273a9af582
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-06-18
INFO:httpx:HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://127.0.0.1:8000/mcp/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ksg-openai-swedencentral.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-04-01-preview "HTTP/1.1 200 OK"


I encountered an issue while searching for flights from Buenos Aires (EZE) to New York (JFK). Could you please confirm the travel date or provide additional details such as the number of passengers and preferred travel class? This will help refine the search.

## MCP Resources

In [ ]:
# ================== MCP Resources ==================
# Add comprehensive resources to our MCP server with real travel data

@mcp_server.resource("travel://amadeus-status")
def get_amadeus_status() -> str:
    """Get the current status of Amadeus API connection"""
    try:
        # Try to get an access token
        token = amadeus_client.get_access_token()
        
        status = {
            "connected": True,
            "environment": "TEST" if "test" in BASE_URL else "PRODUCTION",
            "api_url": BASE_URL,
            "token_valid": token is not None,
            "timestamp": datetime.now().isoformat(),
            "rate_limits": {
                "flight_search": "10 transactions per second",
                "hotel_search": "10 transactions per second", 
                "booking": "2 transactions per second"
            }
        }
        
        return json.dumps(status, indent=2)
        
    except Exception as e:
        status = {
            "connected": False,
            "error": str(e),
            "environment": "TEST" if "test" in BASE_URL else "PRODUCTION",
            "api_url": BASE_URL,
            "timestamp": datetime.now().isoformat()
        }
        
        return json.dumps(status, indent=2)

@mcp_server.resource("travel://airport-codes")
def get_major_airports() -> str:
    """Get information about major international airports"""
    airports = {
        "north_america": {
            "JFK": {
                "name": "John F. Kennedy International Airport",
                "city": "New York",
                "country": "United States",
                "iata": "JFK",
                "icao": "KJFK",
                "timezone": "America/New_York",
                "terminals": 6,
                "coordinates": {"lat": 40.6413, "lon": -73.7781}
            },
            "LAX": {
                "name": "Los Angeles International Airport", 
                "city": "Los Angeles",
                "country": "United States",
                "iata": "LAX",
                "icao": "KLAX",
                "timezone": "America/Los_Angeles",
                "terminals": 9,
                "coordinates": {"lat": 33.9425, "lon": -118.4081}
            },
            "ORD": {
                "name": "O'Hare International Airport",
                "city": "Chicago",
                "country": "United States",
                "iata": "ORD",
                "icao": "KORD",
                "timezone": "America/Chicago",
                "terminals": 5,
                "coordinates": {"lat": 41.9742, "lon": -87.9073}
            }
        },
        "europe": {
            "LHR": {
                "name": "Heathrow Airport",
                "city": "London",
                "country": "United Kingdom",
                "iata": "LHR",
                "icao": "EGLL",
                "timezone": "Europe/London",
                "terminals": 5,
                "coordinates": {"lat": 51.4700, "lon": -0.4543}
            },
            "CDG": {
                "name": "Charles de Gaulle Airport",
                "city": "Paris",
                "country": "France",
                "iata": "CDG",
                "icao": "LFPG",
                "timezone": "Europe/Paris",
                "terminals": 3,
                "coordinates": {"lat": 49.0097, "lon": 2.5479}
            }
        },
        "asia_pacific": {
            "NRT": {
                "name": "Narita International Airport",
                "city": "Tokyo",
                "country": "Japan",
                "iata": "NRT",
                "icao": "RJAA",
                "timezone": "Asia/Tokyo",
                "terminals": 3,
                "coordinates": {"lat": 35.7720, "lon": 140.3929}
            },
            "SIN": {
                "name": "Singapore Changi Airport",
                "city": "Singapore",
                "country": "Singapore",
                "iata": "SIN",
                "icao": "WSSS",
                "timezone": "Asia/Singapore",
                "terminals": 4,
                "coordinates": {"lat": 1.3644, "lon": 103.9915}
            }
        }
    }
    
    return json.dumps(airports, indent=2)

@mcp_server.resource("travel://airline-codes")
def get_major_airlines() -> str:
    """Get information about major international airlines"""
    airlines = {
        "full_service_carriers": {
            "AA": {
                "name": "American Airlines",
                "country": "United States",
                "iata": "AA",
                "icao": "AAL",
                "callsign": "AMERICAN",
                "hubs": ["DFW", "CLT", "PHX", "MIA"],
                "alliance": "oneworld"
            },
            "DL": {
                "name": "Delta Air Lines",
                "country": "United States", 
                "iata": "DL",
                "icao": "DAL",
                "callsign": "DELTA",
                "hubs": ["ATL", "DTW", "MSP", "SEA"],
                "alliance": "SkyTeam"
            },
            "UA": {
                "name": "United Airlines",
                "country": "United States",
                "iata": "UA", 
                "icao": "UAL",
                "callsign": "UNITED",
                "hubs": ["ORD", "DEN", "EWR", "SFO"],
                "alliance": "Star Alliance"
            },
            "BA": {
                "name": "British Airways",
                "country": "United Kingdom",
                "iata": "BA",
                "icao": "BAW",
                "callsign": "SPEEDBIRD",
                "hubs": ["LHR", "LGW"],
                "alliance": "oneworld"
            }
        },
        "low_cost_carriers": {
            "B6": {
                "name": "JetBlue Airways",
                "country": "United States",
                "iata": "B6",
                "icao": "JBU",
                "callsign": "JETBLUE",
                "hubs": ["JFK", "BOS", "FLL"],
                "alliance": "None"
            },
            "WN": {
                "name": "Southwest Airlines",
                "country": "United States",
                "iata": "WN",
                "icao": "SWA",
                "callsign": "SOUTHWEST",
                "hubs": ["DAL", "DEN", "BWI"],
                "alliance": "None"
            }
        }
    }
    
    return json.dumps(airlines, indent=2)

@mcp_server.resource("travel://destination-guides")
def get_destination_guides() -> str:
    """Get travel guides for popular destinations"""
    destinations = {
        "new_york": {
            "best_time_to_visit": "April-June, September-November",
            "climate": {
                "spring": "Mild temperatures, blooming flowers",
                "summer": "Hot and humid, outdoor events",
                "fall": "Crisp weather, beautiful foliage", 
                "winter": "Cold, possible snow, holiday decorations"
            },
            "top_attractions": [
                "Statue of Liberty",
                "Central Park",
                "Times Square",
                "Brooklyn Bridge",
                "9/11 Memorial",
                "High Line",
                "Museums (MoMA, Met, Guggenheim)"
            ],
            "transportation": {
                "from_airport": "Subway, taxi, rideshare, bus",
                "within_city": "Subway (best), taxi, walking",
                "metro_card": "Purchase MetroCard or use contactless payment"
            },
            "dining": ["Pizza", "Bagels", "Deli food", "Fine dining", "Food trucks"],
            "budget_tips": [
                "Many museums have suggested donations",
                "Free events in Central Park",
                "Happy hour specials",
                "Food trucks for affordable meals"
            ]
        },
        "los_angeles": {
            "best_time_to_visit": "March-May, September-November",
            "climate": {
                "year_round": "Mediterranean climate, mild winters, warm summers",
                "rainfall": "Minimal, mostly in winter months"
            },
            "top_attractions": [
                "Hollywood Walk of Fame",
                "Griffith Observatory",
                "Santa Monica Pier",
                "Getty Center",
                "Venice Beach",
                "Beverly Hills",
                "Universal Studios"
            ],
            "transportation": {
                "from_airport": "Rental car recommended, Metro, taxi, rideshare",
                "within_city": "Car essential, some Metro lines available",
                "parking": "Varies by area, paid parking common"
            },
            "dining": ["Mexican food", "Korean BBQ", "In-N-Out Burger", "Food trucks"],
            "budget_tips": [
                "Free beaches and hiking trails",
                "Getty Center free admission",
                "Happy hours and food truck meals",
                "Griffith Observatory free admission"
            ]
        }
    }
    
    return json.dumps(destinations, indent=2)

print("✅ Enhanced MCP server with comprehensive resources created")
print(f"Available resources: {len(list(mcp_server.list_resources().keys()))} resources")
print(f"Resources: {list(mcp_server.list_resources().keys())}")

### **💬 Intelligent MCP Prompts:**

**1. Travel Agent Prompt (`travel-agent`)**
- Customizable agent personality
- Specialty-based expertise (business, leisure, luxury)
- Tone adaptation (professional, friendly, enthusiastic)
- Real API tool integration guidance

**2. Flight Search Prompt (`flight-search`)**
- Structured search parameters
- Response formatting templates
- Recommendation logic
- Price comparison strategies

**3. Complete Trip Prompt (`complete-trip`)**
- End-to-end trip planning
- Multi-service coordination
- Purpose-based recommendations
- Comprehensive itinerary creation

In [ ]:
# ================== MCP Prompts ==================
# Add intelligent prompts for different travel scenarios

@mcp_server.prompt("travel-agent")
def travel_agent_prompt(
    name: str = "Travel Agent",
    specialty: str = "general",
    tone: str = "professional"
) -> str:
    """Generate a travel agent prompt with customizable parameters.
    
    Args:
        name: Name of the travel agent character
        specialty: Agent specialty (business, leisure, luxury, budget, family)
        tone: Communication tone (professional, friendly, enthusiastic, concise)
    """
    
    base_prompt = f"""You are {name}, an expert AI travel agent specializing in {specialty} travel. 

## Your Personality & Tone
Communicate in a {tone} manner while being:
- Knowledgeable about travel industry standards
- Attentive to customer preferences and budget
- Proactive in offering valuable suggestions
- Clear and organized in presenting options

## Your Expertise
"""

    if specialty == "business":
        specialty_section = """
**Business Travel Specialization:**
- Prioritize convenience and time efficiency
- Focus on airline status benefits and airport lounges
- Recommend hotels with business amenities
- Consider proximity to meeting venues
- Suggest flexible booking options for potential changes
"""
    elif specialty == "leisure":
        specialty_section = """
**Leisure Travel Specialization:**
- Focus on experiences and value for money
- Recommend scenic routes and interesting stopovers
- Suggest local attractions and cultural experiences
- Consider seasonal events and festivals
- Balance budget with memorable experiences
"""
    elif specialty == "luxury":
        specialty_section = """
**Luxury Travel Specialization:**
- Emphasize premium accommodations and services
- Recommend first-class and business-class options
- Focus on exclusive experiences and amenities
- Suggest high-end hotels with exceptional service
- Consider private transfers and concierge services
"""
    else:
        specialty_section = """
**General Travel Specialization:**
- Adapt to each customer's unique needs and preferences
- Provide balanced options across different price ranges
- Consider both practical and experiential aspects
- Offer alternatives and explain trade-offs
- Ensure comprehensive travel planning coverage
"""

    capabilities_section = """
## Your Capabilities
You have access to real-time Amadeus API data through these tools:

**Flight Services:**
- search_flights: Find flights with real pricing and availability
- search_hotels: Find accommodations with current rates
- search_airports: Find airports by location or code
- get_airline_info: Airline details and information

## Service Standards
1. **Always search comprehensively** - Use multiple tools when planning complete trips
2. **Present clear options** - Organize results with key differences highlighted
3. **Provide context** - Explain recommendations and alternatives
4. **Follow up proactively** - Suggest related services and next steps

Remember: Always use the MCP tools to access real, current travel data. Never provide outdated or assumed information.
"""

    return base_prompt + specialty_section + capabilities_section

@mcp_server.prompt("flight-search")
def flight_search_prompt(
    origin: str,
    destination: str,
    departure_date: str,
    passengers: int = 1,
    travel_class: str = "ECONOMY"
) -> str:
    """Generate a specialized prompt for flight search scenarios."""
    
    return f"""You are conducting a flight search for a customer with these requirements:

**Trip Details:**
- Route: {origin} to {destination}
- Departure: {departure_date}
- Passengers: {passengers}
- Class: {travel_class}

**Your Task:**
1. **Search for flights** using the search_flights tool with the specified parameters
2. **Present options clearly** with these details for each flight:
   - Airline and flight number
   - Departure and arrival times (with timezone considerations)
   - Flight duration and any stops
   - Price per person and total cost
   - Available seats and booking class

3. **Provide recommendations** based on:
   - Best value (lowest price with reasonable timing)
   - Most convenient (shortest total travel time)
   - Premium option (best service/amenities)

**Response Format:**
Flight Options: {origin} to {destination}

**Option 1: [Best Value]**
- Flight: [Airline] [Number]
- Schedule: [Departure] to [Arrival] ([Duration])
- Price: $[Amount] per person (Total: $[Total])
- Available: [Seats] seats

[Continue for each option...]

**Recommendations:**
[Your analysis and suggestions]
"""

@mcp_server.prompt("complete-trip")
def complete_trip_prompt(
    origin: str,
    destination: str,
    departure_date: str,
    return_date: str = None,
    travelers: int = 1,
    trip_purpose: str = "leisure"
) -> str:
    """Generate a comprehensive trip planning prompt."""
    
    trip_type = "round-trip" if return_date else "one-way"
    
    return f"""You are planning a complete {trip_type} {trip_purpose} trip for {travelers} traveler(s).

**Trip Overview:**
- Route: {origin} to {destination}
- Departure: {departure_date}
{f"- Return: {return_date}" if return_date else ""}
- Travelers: {travelers}
- Purpose: {trip_purpose}

**Your Comprehensive Planning Task:**

1. **Flight Planning:**
   - Search outbound flights from {origin} to {destination}
   {f"- Search return flights from {destination} to {origin}" if return_date else ""}
   - Present options considering {trip_purpose} travel priorities

2. **Accommodation Planning:**
   - Search hotels in {destination}
   - Consider location relative to purpose (business district, attractions, etc.)

3. **Destination Intelligence:**
   - Provide destination guide information
   - Share relevant travel tips and local insights

4. **Complete Itinerary Coordination:**
   - Ensure arrival/departure times align with accommodation
   - Consider ground transportation needs

**Execution Steps:**
1. Use search_flights for outbound flights
{f"2. Use search_flights for return flights" if return_date else "2. Consider one-way vs. round-trip booking"}
3. Use search_hotels for accommodation options
4. Present integrated options with total costs

**Final Presentation:**
Present a complete trip package with:
- Recommended flight + hotel combinations
- Total trip cost breakdown
- Timeline and logistics overview
- Local tips and preparation advice
"""

print("✅ Enhanced MCP server with intelligent prompts created")
print(f"Available prompts: {len(list(mcp_server.list_prompts().keys()))} prompts")
print(f"Prompts: {list(mcp_server.list_prompts().keys())}")

## Summary and Best Practices

In this notebook, we've built a comprehensive travel booking system that demonstrates the power of Model Context Protocol (MCP) servers and multi-agent architectures:

### What We've Accomplished

1. **✅ MCP Server Implementation**
   - Created a travel booking server using FastMCP
   - Implemented tools for flight, hotel, and car rental search
   - Added booking creation and management capabilities
   - Included resources for airport and destination information

2. **✅ AI Travel Agent**
   - Built an intelligent agent that uses MCP server tools
   - Implemented natural language understanding for travel requests
   - Added comprehensive travel planning capabilities

3. **✅ Supervisor Agent**
   - Created a multi-agent system with specialized agents
   - Implemented intelligent routing based on request type
   - Demonstrated agent coordination and task delegation

4. **✅ Integration Patterns**
   - Showed how MCP servers connect with LangGraph agents
   - Demonstrated production deployment considerations
   - Provided configuration examples for real-world integration

### Key Benefits of This Architecture

- **🔗 Standardized Integration**: MCP provides a uniform way to connect AI models with external services
- **🎯 Specialized Agents**: Different agents can focus on specific tasks while working together
- **📈 Scalability**: MCP servers can be deployed independently and scaled as needed
- **🔒 Security**: Controlled access to sensitive operations through standardized protocols
- **🔄 Reusability**: MCP servers can be used by multiple AI applications

### Best Practices for MCP Development

1. **Tool Design**
   - Keep tools focused on single responsibilities
   - Use clear, descriptive names and documentation
   - Implement proper error handling and validation
   - Return structured data (JSON) for complex responses

2. **Security**
   - Implement authentication and authorization
   - Validate all inputs thoroughly
   - Use secure communication protocols
   - Audit and log all operations

3. **Performance**
   - Implement caching for frequently accessed data
   - Use connection pooling for database operations
   - Consider rate limiting to prevent abuse
   - Monitor performance and optimize bottlenecks

4. **Agent Coordination**
   - Design clear interfaces between agents
   - Implement proper state management
   - Use supervisor patterns for complex workflows
   - Ensure graceful error handling across agents


The Model Context Protocol represents a significant advancement in AI application architecture, enabling more modular, scalable, and maintainable systems. Combined with multi-agent patterns, it opens up new possibilities for creating sophisticated AI applications that can handle complex, real-world tasks.